In [ ]:
import geojson
import pandas as pd
import numpy as np
import geopandas
from geopy import distance
import plotly.express as px
import collections
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
from geopy.distance import geodesic
import statsmodels.api as smlog
from sklearn.model_selection import train_test_split
import warnings

In [ ]:
%matplotlib inline
warnings.filterwarnings('ignore')

In [ ]:
ibague_loc = [4.436418, -75.203946]

In [ ]:
def count_nan(df):
    nan_total = df.isna().sum().sum()
    print('En total hay', nan_total)
    print('')
    for col in df:
        if df[col].isna().sum().sum() != 0:
            nan_col = df[col].isna().sum().sum()
            print(col, ' tiene en total: ', nan_col, 'nan')

In [ ]:
def as_percentage(df, variables):
    sub_df = df.copy()
    sub_df['sum'] = sub_df[variables].apply(sum, axis = 'columns')
    for col in sub_df:
        if col in variables:
            name = col + '_pct'
            sub_df[name] = sub_df[col] / sub_df['sum']
    sub_df.replace(np.nan, 0, inplace=True)
    return sub_df

In [ ]:
def find_block(request):
    distance_min = 10000
    for i in range (0, len(ibague_mzn.index)):
        loc_block = (ibague_mzn['LATITUD'].iloc[i],ibague_mzn['LONGITUD'].iloc[i])
        distance = geodesic(loc_block, request).km
        if distance < distance_min:
            distance_min = distance
            closest_block = i
    loc = ibague_mzn['LATITUD'][closest_block],ibague_mzn['LONGITUD'][closest_block] 
    return closest_block

# Data engineer

In [ ]:
location_zonas=pd.read_csv("data/ubicacion_zonasvibra.csv", sep=",")
connections_zonas=pd.read_csv("data/Conexiones_Zonas.csv")
ibague_mzn = geopandas.read_file("data/ibague_manzanas.geojson", driver="GeoJson")

In [ ]:
connections_zonas = connections_zonas.rename(columns={'ene-21': 'enero', 'feb-21': 'febrero', 'mar-21': 'marzo', 'abr-21': 'abril'})
connections_zonas['2021'] = connections_zonas[['enero', 'febrero', 'marzo', 'abril']].apply(sum, axis = 'columns')
connections_zonas = connections_zonas[['ID', 'enero', 'febrero', 'marzo', 'abril', '2021']]

In [ ]:
ibague_mzn = ibague_mzn.drop(columns=['dist_zona1','dist_zona2','dist_zona3','dist_zona4','dist_zona5','dist_zona6','dist_zona7','dist_zona8','dist_zona9','dist_zona10','dist_zona11','dist_zona12','dist_zona13','dist_zona14','dist_zona15','dist_zona16','dist_zona17','dist_zona18','dist_zona19','dist_zona20','dist_zona21','dist_zona22','dist_zona23','dist_zona24','dist_zona25','dist_zona26','dist_zona27','dist_zona28','dist_zona29','dist_zona30','dist_zona31','dist_zona32','dist_zona33','dist_zona34','dist_zona35','dist_zona36','dist_zona37','dist_zona38','dist_zona39','dist_zona40','dist_zona41','dist_zona42','dist_zona43','dist_zona44','dist_zona45','dist_zona46','dist_zona47','dist_zona48','dist_zona49','dist_zona50','dist_zona51','dist_zona52','dist_zona53','dist_zona54','dist_zona55','dist_zona56','dist_zona57','dist_zona58','dist_zona59','dist_zona60','dist_zona61','dist_zona62','dist_zona63','dist_zona64','dist_zona65','dist_zona66','dist_zona67','dist_zona68','dist_zona69','dist_zona70','dist_zona71','dist_zona72','dist_zona73','dist_zona74','dist_zona75','dist_zona76','dist_zona77','dist_zona78','dist_zona79','dist_zona80','dist_zona81','dist_zona82','dist_zona83','dist_zona84','dist_zona85','dist_zona86','dist_zona87','dist_zona88','dist_zona89','dist_zona90','dist_zona91','dist_zona92','dist_zona93','dist_zona94','dist_zona95','dist_zona96','dist_zona97','dist_zona98','dist_zona99','dist_zona100','dist_zona101','dist_zona102','dist_zona103','dist_zona104','dist_zona105','dist_zona106','dist_zona107','dist_zona108','dist_zona109','dist_zona110','dist_zona111','dist_zona112','dist_zona113','dist_zona114','dist_zona115','dist_zona116','dist_zona117','dist_zona118','dist_zona119','dist_zona120','dist_zona121','dist_zona122','dist_zona123','dist_zona124','dist_zona125','dist_zona126','dist_zona127','dist_zona128','dist_zona129','dist_zona130','dist_zona131','dist_zona132','dist_zona133','dist_zona134','dist_zona135','dist_zona136','dist_zona137','dist_zona138','dist_zona139','dist_zona140','dist_zona141','dist_zona142','dist_zona143','dist_zona144','dist_zona145','dist_zona146','dist_zona147','dist_zona148','dist_zona149','dist_zona150'])

In [ ]:
ibague_mzn['dist_min'] = 0
ibague_mzn['zona_min'] = 0

for manzana in ibague_mzn.index:
    
    manzana_min = 0
    minima = 1000000
    loc_manzana = (ibague_mzn['LATITUD'].iloc[manzana], ibague_mzn['LONGITUD'].iloc[manzana])
    
    for i in range (0, len(location_zonas.index)):
        loc_zona = (location_zonas['Latitud'][i],location_zonas['Longitud'][i])
        distance = geodesic(loc_zona, loc_manzana).km
        if distance < minima:
            minima = distance
            manzana_min = i+1
            
    ibague_mzn['dist_min'].iloc[manzana] = minima
    ibague_mzn['zona_min'].iloc[manzana] = manzana_min

In [ ]:
connections_zonas = connections_zonas[['ID', 'enero', 'febrero', 'marzo', 'abril', '2021']].rename(columns={'ID': 'zona_min'})
ibague_mzn = pd.merge(ibague_mzn, connections_zonas, on="zona_min")

In [ ]:
list(ibague_mzn.columns)

In [ ]:
disscard = [
 'DPTO_CCDGO',
 'MPIO_CCDGO',
 'MPIO_CDPMP',
 'SETR_CCDGO',
 'SETR_CCNCT',
 'SECR_CCDGO',
 'SECR_CCNCT',
 'ZU_CCDGO',
 'ZU_CDIVI',
 'SETU_CCDGO',
 'SETU_CCNCT',
 'SECU_CCDGO',
 'SECU_CCNCT',
 'AG_CCDGO',
 'DATO_ANM',
 'VERSION',
 'CTNENCUEST',
 'TP3_1_SI',
 'TP3_2_NO',
 'TP3A_RI',
 'TP3B_TCN',
 'TP4_1_SI',
 'TP4_2_NO',
'CD_LC_CM',
'NMB_LC_CM',
'TP_LC_CM']

ibague_mzn = ibague_mzn.drop(columns=disscard)

In [ ]:
ibague_mzn['pp2021'] = ibague_mzn['2021'] / ibague_mzn['TP27_PERSO']
ibague_mzn['eneropp'] = ibague_mzn['enero'] / ibague_mzn['TP27_PERSO']
ibague_mzn['febreropp'] = ibague_mzn['febrero'] / ibague_mzn['TP27_PERSO']
ibague_mzn['marzopp'] = ibague_mzn['marzo'] / ibague_mzn['TP27_PERSO']
ibague_mzn['abrilpp'] = ibague_mzn['abril'] / ibague_mzn['TP27_PERSO']
ibague_mzn.replace(np.inf, 0, inplace=True)

In [ ]:
count_nan(ibague_mzn)

In [ ]:
ibague_mzn[ibague_mzn['pp2021'].isna()].zona_min.value_counts()

In [ ]:
ibague_mzn = ibague_mzn[ibague_mzn['zona_min'] != 133]

In [ ]:
count_nan(ibague_mzn)

In [ ]:
education = ['TP51PRIMAR', 'TP51SECUND', 'TP51SUPERI', 'TP51POSTGR', 'TP51_13_ED']
estrato = ['TP19_EE_E1', 'TP19_EE_E2', 'TP19_EE_E3', 'TP19_EE_E4', 'TP19_EE_E5', 'TP19_EE_E6']
sexo = ['TP32_1_SEX', 'TP32_2_SEX']
edad = ['TP34_1_EDA', 'TP34_2_EDA', 'TP34_3_EDA', 'TP34_4_EDA', 'TP34_5_EDA', 'TP34_6_EDA', 'TP34_7_EDA', 'TP34_8_EDA', 'TP34_9_EDA']
energia = ['TP19_EE_1', 'TP19_EE_2']
acueducto = ['TP19_ACU_1', 'TP19_ACU_2']
alcantarillado = ['TP19_ALC_1', 'TP19_ALC_2']
gas = ['TP19_GAS_1', 'TP19_GAS_2']
basuras = ['TP19_RECB1', 'TP19_RECB2']
internet = ['TP19_INTE1', 'TP19_INTE2']

In [ ]:
ibague_mzn = as_percentage(ibague_mzn, education)
ibague_mzn = as_percentage(ibague_mzn, estrato)
ibague_mzn = as_percentage(ibague_mzn, sexo)
ibague_mzn = as_percentage(ibague_mzn, edad)
ibague_mzn = as_percentage(ibague_mzn, energia)
ibague_mzn = as_percentage(ibague_mzn, acueducto)
ibague_mzn = as_percentage(ibague_mzn, alcantarillado)
ibague_mzn = as_percentage(ibague_mzn, gas)
ibague_mzn = as_percentage(ibague_mzn, basuras)
ibague_mzn = as_percentage(ibague_mzn, internet)

# Modelling

In [ ]:
distance_threshold = 0.4
ibague_mzn['far_dist'] = [1 if x > distance_threshold else 0 for x in ibague_mzn['dist_min']]
ibague_mzn['far_dist'].value_counts(normalize = True)

In [ ]:
ibague_mzn = ibague_mzn[['far_dist', 'DENSIDAD','febrero','marzo','abril','TP19_EE_E2_pct','TP19_EE_E3_pct','TP19_EE_E4_pct',
                         'TP19_EE_E5_pct','TP19_EE_E6_pct','TP34_2_EDA_pct','TP34_3_EDA_pct','TP34_4_EDA_pct','TP34_5_EDA_pct',
                         'TP34_6_EDA_pct','TP34_7_EDA_pct','TP34_8_EDA_pct','TP34_9_EDA_pct','TP19_ALC_1_pct','TP19_EE_1_pct',
                         'TP19_GAS_1_pct','TP19_INTE2_pct', 'geometry', 'LATITUD', 'LONGITUD']]

In [ ]:
ibague_mzn['Intercept'] = 1
y, X = ibague_mzn['far_dist'], ibague_mzn[['DENSIDAD','febrero','marzo','abril','TP19_EE_E2_pct','TP19_EE_E3_pct','TP19_EE_E4_pct','TP19_EE_E5_pct','TP19_EE_E6_pct','TP34_2_EDA_pct','TP34_3_EDA_pct','TP34_4_EDA_pct','TP34_5_EDA_pct','TP34_6_EDA_pct','TP34_7_EDA_pct','TP34_8_EDA_pct','TP34_9_EDA_pct','TP19_ALC_1_pct','TP19_EE_1_pct','TP19_GAS_1_pct','TP19_INTE2_pct']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
logit = smlog.Logit(y_train, X_train).fit()
logit.summary()

In [ ]:
score_logit = logit.predict(X)

In [ ]:
ibague_mzn['prob_logit'] = logit.predict(X)

In [ ]:
request = (4.442944852223881, -75.16375287055769)

In [ ]:
closest_block = find_block(request)
ibague_mzn.iloc[closest_block]['prob_logit']

In [ ]:
lats=ibague_mzn["LATITUD"] 
lons=ibague_mzn["LATITUD"]

fig = px.choropleth_mapbox(ibague_mzn, geojson=ibague_mzn.geometry, 
                    locations=ibague_mzn.index, color="prob_logit",
                    height=600,
                    color_continuous_scale="Greens",
                    labels={'prob_logit':'Far from 400mts to a Wifi Zone'},
                    mapbox_style='carto-positron')

fig.add_scattermapbox( lat=lats, lon=lons, 
                      mode = 'markers', 
                      marker_size=12,
                      marker_color='#167AC6', hoverlabel={ })

fig.update_layout(mapbox_zoom = 12, 
                  height=460,
                  mapbox_center = {"lat": 4.435800, "lon": -75.199009})

fig.update_layout(transition_duration=500)
fig['layout'].update(margin=dict(l=0,r=0,b=0,t=0))

fig.show()